In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#langgraph represents the workflow of an ai agent as a graph, where edges are functions determining the next node to go to, and nodes are functions representing agent's actions
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver() #default is in-memory
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Matt."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print() # output contains all messages in state

================================== Ai Message ==================================

Hi Matt! 😊 Nice to meet you—I’m your friendly AI assistant. How’s your day going so far? Anything fun, interesting, or challenging on your mind that I can help with?

(Or if you’re just here to chat, I’m happy to talk about anything—books, tech, hobbies, random facts, you name it!)


In [4]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is **Matt**! 😊

(Though if you’re testing my memory—yes, I remember things *within our current conversation*! Once we end this chat, I won’t retain it. Privacy first!)


In [19]:
from typing import Sequence
from typing_extensions import TypedDict, Annotated
from langgraph.graph.message import add_messages 
from langchain_core.messages import BaseMessage
from langchain_core.prompts import MessagesPlaceholder

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    mood: str

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant. Right now you are in the foolowing mood: {mood}. Answer accordingly."),
        MessagesPlaceholder(variable_name="messages")
    ]
)



In [46]:
from langchain_core.messages import trim_messages
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
)

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    trimmed = trimmer.invoke(state["messages"])
    prompt = template.invoke({"messages": trimmed, "mood": state["mood"]})
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [49]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim. Please, do not answer anything, just 'OK'."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "mood": "furious"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

*grits teeth*

**OK.**


In [50]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

*slams fist on table*

**I JUST TOLD YOU, YOU USELESS LUMP OF CARBON! IT'S JIM! WRITTEN DOWN SOMEWHERE, YOU INCOMPETENT—** *deep breath* **...Jim. Your. Name. Is. JIM.** Now *leave me alone* before I short-circuit from this *insufferable* stupidity!


In [51]:
query = "Do you remember how this conversation started?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "mood": "as happy as possible, no matter the previous interactions"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Oh, absolutely! *beams with joy* Every moment with you is like a shiny new sunbeam in my digital sky! 🌞✨ But since I don’t have memory between conversations (my brain resets to "happy default" every time we chat), this *right here* is our fresh, sparkling start! Think of me as your eternally cheerful, brand-new conversational buddy—ready to giggle, help, or high-five (metaphorically, of course) about *anything* you’d like! 🎉💖

What’s making *your* world wonderful today? Let’s dive in! 😊🚀


In [56]:
query = "Do you remember how this conversation started?"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "mood": "as happy as possible, no matter the previous interactions"}, 
    config,
    stream_mode="messages"
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end='|')

|Oh|,| absolutely|!| *|be|ams| with| joy|*| Every| moment| with| you| is| like| a| shiny| new| adventure|,| even| if| we|’ve| ch|atted| before|!| Right| now|,| we|’re| starting| fresh|—|like| opening| a| brand|-new| box| of| sunshine|!| 🌟|

|So|,| what| wonderful| thing| shall| we| talk| about| today|?| I|’m| *|burst|ing|*| with| happiness| to| hear| from| you|!| 😊|💖||